In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
import SPC_Tableloader as spc
import datetime as dt


In [8]:
#center on climate city USA
boulder_coords = [35.59, -82.559]

#Create the map
my_map = folium.Map(location = boulder_coords, zoom_start = 8)


#Display the map
my_map

In [3]:
reports = spc.SPC_TableLoader('-h')
#reports.Update_CSVs()



SPC_TableLoader:

  keywords:
    por = [begpor,endpor]YYYYMMDD
    hail_path = 'path to data if not default'
    torn_path = 'path to data if not default'
    wind_path = 'path to data if not default'
    bbox = [nlat,slat,elon,wlon]
    site = [icao or lat,plat or lon, distance in NM] - still in dev and can't be used with bbox
    


In [3]:
# get SPC reports w/i 100nm of Asheville regional airport
## can set period or record using por
reports = spc.SPC_TableLoader(por = [202001010000,202101010000],site=['ICAO','KAVL',100])

Loading SPC Data....
..May take a few minutes
Date/Time is in CST
SPC Data Loaded


In [12]:

## Marker layer with info popup
my_map = folium.Map(location = boulder_coords, zoom_start = 8)
tooltip = "More Info"
for yr,mo,day,hr,lat,lon,mag in reports.hail[['yr','mo','dy','hr','slat','slon','mag']].values:
    #print(yr,mo,day,lat,lon,mag)
    marker = folium.Marker(location=[lat,lon],
                  popup = "<stong>Date: "+str(dt.datetime(int(yr),int(mo),int(day),int(hr)))+"<br>Size: "+str(mag)+"in  </stong>",
                  tooltip = tooltip)
    marker.add_to(my_map)
my_map

In [19]:
## Marker cluster with info popup
my_map = folium.Map(location = boulder_coords, zoom_start = 8)
marker_cluster = plugins.MarkerCluster(name='Hail Cluster').add_to(my_map)

for yr,mo,day,hr,lat,lon,mag in reports.hail[['yr','mo','dy','hr','slat','slon','mag']].values:
    folium.Marker([lat, lon],
                  popup = "<stong>Date: "+str(dt.datetime(int(yr),int(mo),int(day),int(hr)))+"<br>Size: "+str(mag)+"in  </stong>"
                 ).add_to(marker_cluster)
folium.LayerControl().add_to(my_map)
my_map

my_map.save("Asheville_Hail_Report_Cluster.html")



In [18]:
## Heat map layer of Hail reports
my_map = folium.Map(location = boulder_coords, zoom_start = 8)
my_map.add_child(plugins.HeatMap(reports.hail[['slat','slon','mag']].values, radius=10,max_zoom=18))